In [27]:
import opentrons as ot
from opentrons import protocol_api

metadata = {'apiLevel': '2.13'}

In [28]:
from opentrons import simulate
protocol = simulate.get_protocol_api(metadata['apiLevel']) # uncomment to run in simulation

#protocol = opentrons.execute.get_protocol_api(metadata['apiLevel']) # uncomment to run on the actual hardware


/Users/omelkonyan/.opentrons/robot_settings.json not found. Loading defaults
/Users/omelkonyan/.opentrons/deck_calibration.json not found. Loading defaults


In [29]:
# Load labware
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 2)
block = plate #protocol.load_labware('xxx', )
tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', 1)
pipette = protocol.load_instrument('p300_single', 'right',
                                     tip_racks=[tiprack])

In [30]:
def serial_dilution(dye, water, destination_wells, dilution_factor=0.5, volume=50):
    vol_water = (1-dilution_factor) * volume
    vol_dye = dilution_factor * volume
    
    pipette.pick_up_tip() 
    
    # Per-fill with water
    for well in destination_wells:
        pipette.aspirate(vol_water, water)
        pipette.dispense(vol_water, well)
    
    # For each destination well, calcualte where we will take liquid from 
    source_wells = [dye] + destination_wells[:-1]
    print('Source wells:', source_wells)
    
    # Pre-mix dye container
    pipette.mix(3, 150, dye)
    for source_well, dest_well in zip(source_wells, destination_wells):
        pipette.aspirate(vol_dye, source_well)
        pipette.dispense(vol_dye, dest_well)
        pipette.mix(3, 150, dest_well)
    
    pipette.drop_tip()

In [31]:
dest_wells = plate.rows_by_name()['A'][:4]
print('Destination wells:', dest_wells)
serial_dilution(dye=block['A1'], water=block['A2'], destination_wells=dest_wells)

Destination wells: [A1 of Corning 96 Well Plate 360 µL Flat on 2, A2 of Corning 96 Well Plate 360 µL Flat on 2, A3 of Corning 96 Well Plate 360 µL Flat on 2, A4 of Corning 96 Well Plate 360 µL Flat on 2]
Source wells: [A1 of Corning 96 Well Plate 360 µL Flat on 2, A1 of Corning 96 Well Plate 360 µL Flat on 2, A2 of Corning 96 Well Plate 360 µL Flat on 2, A3 of Corning 96 Well Plate 360 µL Flat on 2]
